In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which i

In [2]:
!git config --global user.email "anhnguye@usc.edu"
!git config --global user.name "Andy Nguyen"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [111]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer, pipeline
import torch
from datasets import Dataset, DatasetDict, ClassLabel

# 1. Domain Adaptation

In [10]:
# filename = 'drive/My Drive/preprocessed_comments.csv'
filename = '/preprocessed_comments.csv'

data = pd.read_csv(filename)
print(f'Total rows: {data.shape[0]}\n')

data = data.drop(columns=['Professor ID', 'skip_class', 'IsCourseOnline', 'noTag']) # drop unnecessary cols
data = data.drop(data[data.drop(columns=['Comment', 'Star Rating', 'Course Difficulty']).sum(axis=1) == 0].index) # dropping the 14 rows with no tags
print(f'Rows after dropping no tags: {data.shape[0]}\n')

data = data.dropna(subset=['Comment']) # dropping Nones
data = data[data['Comment'].str.strip() != ""] # dropping ''
print(f'Rows after dropping empty comments: {data.shape[0]}\n')

train, test = train_test_split(data, test_size=0.2, random_state=42)
unsup, sup = train_test_split(train, test_size=0.25, random_state=42)
print(f'Unsupervised rows: {unsup.shape[0]} | Supervised (fine-tuning) rows: {sup.shape[0]} | Test rows: {test.shape[0]}')

Total rows: 34440

Rows after dropping no tags: 34426

Rows after dropping empty comments: 34400

Unsupervised rows: 20640 | Supervised (fine-tuning) rows: 6880 | Test rows: 6880


In [11]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
print(f'{model.num_parameters()} parameters model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

66985530 parameters model


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
# Split 'unsup' into train and test datasets
train_unsup, test_unsup = train_test_split(unsup, test_size=0.1, random_state=42) # Splitting unsup into train and test (9:1 ratio)

# Create Dataset objects
train_dataset = Dataset.from_pandas(train_unsup[['Comment']])
test_dataset = Dataset.from_pandas(test_unsup[['Comment']])

# Create DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

dataset

DatasetDict({
    train: Dataset({
        features: ['Comment', '__index_level_0__'],
        num_rows: 18576
    })
    test: Dataset({
        features: ['Comment', '__index_level_0__'],
        num_rows: 2064
    })
})

In [14]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"Comment: {row['Comment']}\n")

Comment: along pages reviews, negative experience. friendly questions, unclear grading, overwhelming amount work. tests assigned every friday due sunday. test easter break, final exam mothers day. hope school take these reviews consideration, go back far 2002.

Comment: mccoy great professor, especially covid understanding made things feel less stressful. read lot interesting books. write papers books gives lot informationcontext within lectures. 1010 would take another class again.

Comment: choice take another course, take that course this course this professor. syllabus says 3 credit hours course, feel like 10 credit hourse lots homeworks. also, takes forever give back grades, problem estimate final grade befroe final week.



In [15]:
def tokenize_function(examples):
    result = tokenizer(examples['Comment'])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_dataset = dataset.map(
    tokenize_function, batched=True, remove_columns=["Comment", "__index_level_0__"]
)
tokenized_dataset

Map:   0%|          | 0/18576 [00:00<?, ? examples/s]

Map:   0%|          | 0/2064 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 18576
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 2064
    })
})

In [16]:
chunk_size = 128

In [17]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


In [18]:
lm_datasets = tokenized_dataset.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/18576 [00:00<?, ? examples/s]

Map:   0%|          | 0/2064 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5289
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 591
    })
})

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [20]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

print(f'Masked inputs:')
for chunk in data_collator(samples)["input_ids"]:
    print(f"\n{tokenizer.decode(chunk)}")

Masked inputs:

[CLS] take this class [MASK] strong accent cannot lasted what saying [MASK] grades midter [unused583] final well them. [SEP] [CLS] never boring moment this class [SEP] [CLS] take dr. hamdeh want chance understanding making good grade [MASK]. professors wichita state terrible. hamdeh [MASK] teach make sense. review test. also give extra credit attendance low. [SEP] [CLS] ok class.ᅲ work difficult. professor [MASK]man good professor this class. email back fairly quickly questions. midterm final essays. make sure send [MASK] rough [MASK] turning feedback really helps [MASK] [SEP] [CLS] nancy probably one favourite prof [MASK] resentment comes harsh first she [unused770] really funny makes lectures real [MASK] fun [SEP] [CLS] lectures

great [MASK] assignments interesting [MASK] [SEP] [CLS] amazing [MASK] [MASK] definitely take class wasnt graduating [MASK] months!! [SEP] [CLS] amazing prof! [SEP] [CLS] [MASK] intimidating prof ive ever had. [MASK] time yelling [MASK] thing

In [21]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(lm_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-ratemyprof",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [66]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Perplexity: 386.85


In [24]:
for i in range(3):
    trainer.train()
    eval_results = trainer.evaluate()
    print(f"Perplexity (after {i+1} epochs): {math.exp(eval_results['eval_loss']):.2f}")

Epoch,Training Loss,Validation Loss
1,4.931000,4.528771
2,4.459200,4.290766
3,4.349500,4.274695


Perplexity (after 1 epochs): 73.17


Epoch,Training Loss,Validation Loss
1,4.250600,4.130980
2,4.151400,4.036473
3,4.092100,3.989091


Perplexity (after 2 epochs): 57.09


Epoch,Training Loss,Validation Loss
1,4.073500,4.015939
2,3.986800,3.937064
3,3.960300,3.872381


Perplexity (after 3 epochs): 51.20


In [25]:
for i in range(3, 30):
    trainer.train()
    eval_results = trainer.evaluate()
    print(f"Perplexity (after {i+1} epochs): {math.exp(eval_results['eval_loss']):.2f}")

Epoch,Training Loss,Validation Loss
1,3.923600,3.901315
2,3.891100,3.813618
3,3.850100,3.825049


Perplexity (after 4 epochs): 45.44


Epoch,Training Loss,Validation Loss
1,3.861800,3.804756
2,3.821700,3.745920
3,3.775700,3.753550


Perplexity (after 5 epochs): 42.30


Epoch,Training Loss,Validation Loss
1,3.796000,3.762187
2,3.745200,3.745203
3,3.715700,3.701454


...

Perplexity (after 28 epochs): 29.37


Epoch,Training Loss,Validation Loss
1,3.138400,3.386537
2,3.133200,3.325722
3,3.132500,3.389680


Perplexity (after 29 epochs): 29.47


Epoch,Training Loss,Validation Loss
1,3.113700,3.333594
2,3.111400,3.400153
3,3.107600,3.319800


Perplexity (after 30 epochs): 27.47


In [26]:
trainer.push_to_hub()

events.out.tfevents.1732517606.6591239748dc.134.29:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1732517704.6591239748dc.134.30:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/herald-of-spring/distilbert-base-uncased-finetuned-ratemyprof/commit/a0d63b03b249599006151d154dc4f2a14891b512', commit_message='End of training', commit_description='', oid='a0d63b03b249599006151d154dc4f2a14891b512', pr_url=None, pr_revision=None, pr_num=None)

# 2. Fine-Tuning

In [36]:
mask_filler = pipeline("fill-mask", model="herald-of-spring/distilbert-base-uncased-finetuned-ratemyprof")
preds = mask_filler('One of the most [MASK] classes I have ever taken')

for pred in preds:
    print(pred['sequence'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


one of the most challenging classes i have ever taken
one of the most amazing classes i have ever taken
one of the most difficult classes i have ever taken
one of the most inspiring classes i have ever taken
one of the most enjoyable classes i have ever taken


In [119]:
model_checkpoint = "herald-of-spring/distilbert-base-uncased-finetuned-ratemyprof"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1, problem_type='regression')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at herald-of-spring/distilbert-base-uncased-finetuned-ratemyprof and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [120]:
def tokenize_function(examples):
    result = tokenizer(examples['Comment'], padding=True, truncation=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [121]:
# Create Dataset objects
train_star = Dataset.from_pandas(sup[['Comment', 'Star Rating']])
test_star = Dataset.from_pandas(test[['Comment', 'Star Rating']])

# Create DatasetDict
star_data = DatasetDict({
    'train': train_star,
    'test': test_star,
})

star_data

DatasetDict({
    train: Dataset({
        features: ['Comment', 'Star Rating', '__index_level_0__'],
        num_rows: 6880
    })
    test: Dataset({
        features: ['Comment', 'Star Rating', '__index_level_0__'],
        num_rows: 6880
    })
})

In [122]:
star_data = star_data.cast_column("Star Rating", ClassLabel(names=list(range(1,6))))
star_data['train'].features

Casting the dataset:   0%|          | 0/6880 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6880 [00:00<?, ? examples/s]

{'Comment': Value(dtype='string', id=None),
 'Star Rating': ClassLabel(names=[1, 2, 3, 4, 5], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [123]:
tokenized_star = dataset.map(
    tokenize_function, batched=True, remove_columns=["Comment", "__index_level_0__"]
)
tokenized_star

Map:   0%|          | 0/6880 [00:00<?, ? examples/s]

Map:   0%|          | 0/6880 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Star Rating', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 6880
    })
    test: Dataset({
        features: ['Star Rating', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 6880
    })
})

In [124]:
tokenized_star = tokenized_star.rename_column("Star Rating", "labels")
tokenized_star.set_format("torch")
tokenized_star

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 6880
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 6880
    })
})

In [125]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_star["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-star",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [126]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    mse = mean_squared_error(labels, predictions)
    return {"mse": mse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_star["train"],
    eval_dataset=tokenized_star["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [127]:
print('Initial loss:', trainer.evaluate()['eval_loss'])

Initial loss: 14.68386173248291


In [114]:
trainer.train()
print(f"Loss: {trainer.evaluate()['eval_loss']:.2f}")

Epoch,Training Loss,Validation Loss,Model Preparation Time,Mse
1,2.718400,0.746526,0.001700,14.908576
2,0.627800,0.662922,0.001700,14.908576
3,0.489500,0.661218,0.001700,14.908576


Loss: 0.66


In [ ]:
trainer.push_to_hub()

In [44]:
train_diff = Dataset.from_pandas(sup[['Comment', 'Course Difficulty']])
test_diff = Dataset.from_pandas(test[['Comment', 'Course Difficulty']])

diff_data = DatasetDict({
    'train': train_diff,
    'test': test_diff,
})

diff_data

DatasetDict({
    train: Dataset({
        features: ['Comment', 'Course Difficulty', '__index_level_0__'],
        num_rows: 6880
    })
    test: Dataset({
        features: ['Comment', 'Course Difficulty', '__index_level_0__'],
        num_rows: 6880
    })
})

In [45]:
train_tags = Dataset.from_pandas(sup.drop(columns=['Star Rating', 'Course Difficulty']))
test_tags = Dataset.from_pandas(test.drop(columns=['Star Rating', 'Course Difficulty']))

tags_data = DatasetDict({
    'train': train_tags,
    'test': test_tags,
})

tags_data

DatasetDict({
    train: Dataset({
        features: ['Comment', 'gives_good_feedback', 'caring', 'respected', 'participation_matters', 'clear_grading_criteria', 'amazing_lectures', 'inspirational', 'tough_grader', 'hilarious', 'get_ready_to_read', 'lots_of_homework', 'accessible_outside_class', 'lecture_heavy', 'extra_credit', 'graded_by_few_things', 'group_projects', 'would_take_again', 'skip_class_you_wont_pass', 'test_heavy', 'so_many_papers', 'beware_of_pop_quizzes', 'tests_are_tough', '__index_level_0__'],
        num_rows: 6880
    })
    test: Dataset({
        features: ['Comment', 'gives_good_feedback', 'caring', 'respected', 'participation_matters', 'clear_grading_criteria', 'amazing_lectures', 'inspirational', 'tough_grader', 'hilarious', 'get_ready_to_read', 'lots_of_homework', 'accessible_outside_class', 'lecture_heavy', 'extra_credit', 'graded_by_few_things', 'group_projects', 'would_take_again', 'skip_class_you_wont_pass', 'test_heavy', 'so_many_papers', 'beware_of_pop_